In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
import calendar
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus import AbstractRetrieval

from tqdm import tqdm

pd.options.display.max_columns = 30

filename = "./210803/BIPV_ML.txt"
filename_rooftop = "./210803/rooftop_ML.txt"

In [2]:
### language
dic_language = {'eng': "English",
                'kor': "Korean"
               }


In [3]:
df_bipv = pd.read_pickle("./210803/df_bipv.pkl")
print(df_bipv.shape)
df_bipv.head()

(14385, 34)


,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,affiliation_city,affiliation_country,author_count,author_names,author_ids,...,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85110764627,10.1016/j.renene.2021.07.075,S0960148121010806,None,Markov chains estimation of the optimal period...,ar,Article,Sánchez-Barroso G.,60017838;126596986,Universidad de Extremadura;InnoDesarrollo S.L.,Badajoz;Badajoz,Spain;Spain,4,"Sánchez-Barroso, Gonzalo;González-Domínguez, J...",57216405703;57216406595;57197703890;57197479389,...,09601481,27569,18790682,Journal,179,None,None,537-549,The European Dehesa has a very high potential ...,Cleaning frequency | Cleaning photovoltaic sys...,0,1,ESF,PD18047,European Social Fund
1,2-s2.0-85110299293,10.1016/j.energy.2021.121415,S0360544221016637,None,Stochastic modelling of variable renewables in...,ar,Article,Seljom P.,60004205;101126845,SINTEF Foundation for Scientific and Industria...,Trondheim;Kjeller,Norway;Norway,5,"Seljom, Pernille;Kvalbein, Lisa;Hellemo, Lars;...",37108388300;57226090492;26767755100;2043623340...,...,03605442,29348,None,Journal,236,None,121415,None,Variable electricity generation from wind and ...,Energy-system modelling | Satellite-data | Sce...,0,0,None,268097,Norges Forskningsråd
2,2-s2.0-85107321575,10.1038/s41467-021-23592-0,None,34059672,Revealing composition and structure dependent ...,ar,Article,Lian W.,60089931;125236348,Hefei National Laboratory for Physical Science...,Hefei;Hefei,China;China,8,"Lian, Weitao;Jiang, Chenhui;Yin, Yiwei;Tang, R...",57204036372;57202454733;57203962991;5719647276...,...,None,19700182758,20411723,Journal,12,1,3260,None,Antimony trisulfide (Sb2S3) is a kind of emerg...,None,0,1,IUSS,22005293,National Outstanding Youth Science Fund Projec...
3,2-s2.0-85107176116,10.1038/s41598-021-91061-1,None,None,Combined nano and micro structuring for enhanc...,ar,Article,Perrakis G.,60072753;60029857;60001524,Texas A&amp;M University at Qatar;Foundation f...,Doha;Heraklion;Rethymnon,Qatar;Greece;Greece,6,"Perrakis, George;Tasolamprou, Anna C.;Kenanaki...",57206465841;25925551900;21234204900;7004533566...,...,None,21100200805,20452322,Journal,11,1,11552,None,Outdoor devices comprising materials with mid-...,None,0,1,QNRF,NPRP9-383-1-083,Qatar National Research Fund
4,2-s2.0-85104778117,10.1038/s41598-021-87714-w,None,33883574,Decay radius of climate decision for solar pan...,ar,Article,Barton-Henry K.,60159447;60030162;60025182;60025038;60021763,Mercator Research Institute on Global Commons ...,Berlin;New York;Potsdam;Berkeley;Potsdam,Germany;United States;Germany;United States;Ge...,3,"Barton-Henry, Kelsey;Wenz, Leonie;Levermann, A...",57223087322;56422119800;8856583300,...,None,21100200805,20452322,Journal,11,1,8571,None,To design incentives towards achieving climate...,None,0,1,None,undefined,Volkswagen Foundation


In [4]:
df_rooftop = pd.read_pickle("./210803/df_rooftop.pkl")
print(df_rooftop.shape)
df_rooftop.head()

(921, 34)


,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,affiliation_city,affiliation_country,author_count,author_names,author_ids,...,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85110764627,10.1016/j.renene.2021.07.075,S0960148121010806,None,Markov chains estimation of the optimal period...,ar,Article,Sánchez-Barroso G.,60017838;126596986,Universidad de Extremadura;InnoDesarrollo S.L.,Badajoz;Badajoz,Spain;Spain,4,"Sánchez-Barroso, Gonzalo;González-Domínguez, J...",57216405703;57216406595;57197703890;57197479389,...,09601481,27569,18790682,Journal,179,None,None,537-549,The European Dehesa has a very high potential ...,Cleaning frequency | Cleaning photovoltaic sys...,0,1,ESF,PD18047,European Social Fund
1,2-s2.0-85107176116,10.1038/s41598-021-91061-1,None,None,Combined nano and micro structuring for enhanc...,ar,Article,Perrakis G.,60072753;60029857;60001524,Texas A&amp;M University at Qatar;Foundation f...,Doha;Heraklion;Rethymnon,Qatar;Greece;Greece,6,"Perrakis, George;Tasolamprou, Anna C.;Kenanaki...",57206465841;25925551900;21234204900;7004533566...,...,None,21100200805,20452322,Journal,11,1,11552,None,Outdoor devices comprising materials with mid-...,None,0,1,QNRF,NPRP9-383-1-083,Qatar National Research Fund
2,2-s2.0-85108691578,10.1016/j.seta.2021.101320,S2213138821003301,None,Techno-economic optimization for grid-friendly...,ar,Article,Christiaanse T.V.,60032882;60003122,Technische Universiteit Eindhoven;University o...,Eindhoven;Victoria,Netherlands;Canada,3,"Christiaanse, T. V.;Loonen, R. C.G.M.;Evins, R.",57189358262;55504415000;36439398000,...,22131388,21100239262,None,Journal,47,None,101320,None,High levels of non-dispatchable photo-voltaic ...,Bi-level optimization | British Columbia | Ene...,0,0,VU,undefined,Victoria University
3,2-s2.0-85107622660,10.1016/j.enconman.2021.114327,S0196890421005033,None,Coordinated energy-environmental-economic opti...,ar,Article,Hong Y.,60104720;60073652;60022195;60015138;113229129,University of Nottingham Ningbo China;Tongji U...,Ningbo;Shanghai;Cambridge;Nottingham;Shanghai,China;China;United States;United Kingdom;China,7,"Hong, Y.;Ezeh, Collins I.;Deng, W.;Hong, S. H....",57207964999;57194004224;57191907391;5664702470...,...,01968904,29372,None,Journal,243,None,114327,None,Given that energy-efficiency policies focus on...,Life-cycle cost analysis | Low-rise | Office b...,0,0,None,undefined,None
4,2-s2.0-85105000579,10.1016/j.ijepes.2021.106932,S0142061521001721,None,Cool roof coating impact on roof-mounted photo...,ar,Article,Rahmani F.,60011442;126136083,Lamar University;Momentum Bay LLC,Beaumont;Houston,United States;United States,3,"Rahmani, Fatemeh;Robinson, Mark Alan;Barzegara...",57198277655;57223142090;26535502400,...,01420615,17985,None,Journal,130,None,106932,None,Cool-roofing is effective in reducing cooling ...,Cool roof coating | DC-coupled solar photovolt...,0,0,NCI,undefined,National Cancer Institute


In [5]:
columns=["eid", "PT", "AU", "AF", "TI", 
         "SO", "SO_abb", "LA", "DT", "DE", 
         "ID", "AB", "C1", "RP", "EM", 
         "CR", "NR", "TC", "Z9", "SN", 
         "J9", "JI", "PD", "PY", "VL", 
         "AR", "DI", "SC"]

df_ab = pd.DataFrame(columns=columns)

In [6]:
starting_index = 0
data_index = list(range(starting_index, df_bipv.shape[0]))

for art in tqdm(data_index, desc="looping over df_bipv"):

    eid = df_bipv.loc[art, "eid"]
    ab = AbstractRetrieval(eid, view="FULL")

    # 1. PT: publication type
    docu_type_ = ab.srctype

    # 2. AU: index name
    index_name_ = [a.indexed_name for a in ab.authors] if ab.authors else [None]

    # 3. AF: author name
    author_name_ = [f"{a.surname}, {a.given_name}" for a in ab.authors] if ab.authors else [None]

    # 4. TI: document title
    docu_title_ = df_bipv.loc[art, "title"]

    # 5. SO: publication name
    src_title_ = ab.publicationName

    # 5-1. publication abbr.
    src_abb_ = ab.sourcetitle_abbreviation

    # 6. LA : Language
    try:
        language_ = dic_language[ab.language]
    except:
        language_ = "unknown"

    # 7. DT : Document Type
    docu_type_ = ab.subtype

    # 8. DE : Author Keywords
    auth_kw_ = ab.authkeywords

    if not auth_kw_:
        auth_kw_ = 'None'
    else:
        auth_kw_ = '; '.join(auth_kw_)

    # 9. ID : Keyword Plus
    kw_plus_ = 'None'

    # 10. AB : Abstract
    abstract_ = ab.abstract

    # 11. C1 : Author Address
#     if ab.authorgroup:
#         tmp = pd.DataFrame(ab.authorgroup)
#         grouped = tmp.groupby('organization').agg(lambda x: list(x))

#         aff_names = [str(aff) for aff in grouped["affiliation_id"].index.tolist()]
#         aff_ids = grouped["affiliation_id"].tolist()
#         citys = grouped["city"].tolist()
#         countrys = grouped["country"].tolist()
#         auids = grouped["auid"].tolist()
#         indexed_names = grouped["indexed_name"].tolist()

#         address = []
#         for aff_name, aff_id, city, country in zip(aff_names, aff_ids, citys, countrys):
#             if isinstance(aff_id, list):
#                 aff_id, city, country = aff_id[0], city[0], country[0]
#             address.append(f"{aff_id}, {aff_name}, {city}, {country}")

#         addresss_ = list(zip(auids, indexed_names, address))
    if ab.affiliation and ab.authors:
        df_aff = pd.DataFrame(ab.affiliation)
        df_aff["id"] = df_aff["id"].astype(str)
        df_auth = pd.DataFrame(ab.authors)
        df_authgroup = df_auth.groupby("affiliation").agg(list).reset_index()

        df_aff = pd.merge(df_aff, df_authgroup, left_on="id", right_on="affiliation").drop(["affiliation"], axis=1)
        df_aff["address"] = df_aff[['id', 'name', 'city', 'country']].apply(lambda x: ', '.join(x.astype(str)), axis=1)
        addresss_ = df_aff[["auid", "indexed_name", "address"]].values.tolist()
    else:
        addresss_ = []

    # df_aff = pd.DataFrame(ab.authorgroup)
    # grouped = df_aff.groupby("affiliation_id")
    # grouped[["organization", "affiliation_id", "city", "country"]]

    # 12. RP : Reprint Address
    rep_addr_ = "None"

    # 13. EM : E-mail Address
    em_addr_ = "None"

    # 14. CR : Cited References
    refs_ = []
    if ab.references != None:
        tmp = pd.DataFrame(ab.references)
        refcount = int(ab.refcount)

        for i in range(refcount):
            tmp_ = tmp.iloc[i]
            tmp_authors = tmp_['authors']
            if tmp_authors == None:
                tmp_authors = "[Anonymous]"
            tmp_year = tmp_['publicationyear']
            tmp_src = tmp_['sourcetitle']
            tmp_vol = tmp_['volume']
            tmp_page = tmp_['first']
            tmp_doi = tmp_['doi']

            ref = tmp_authors
            for item in [tmp_year, tmp_src, tmp_vol, tmp_page]:
                if item != None:
                    ref = ', '.join([ref, item])
            if tmp_doi != None:
                ref = ref + f", DOI {tmp_doi}"

            if i == 0:
                refs_.append(f"CR {ref}")
            else:
                refs_.append(f"   {ref}")

    # 15. NR : Cited Reference Count
    nr_ = ab.refcount

    # 16. TC : Web of Science Core Collection Times Cited Count
    tc_ = ab.citedby_count

    # 17. Z9 : Total Times Cited Count
    cc_ = tc_

    # 18. U1 : Usage Count (Last 180 Days)
    # 19. U2 : Usage Count (Since 2013)
    # 20. PU : Publisher = ELSEVIER SCI LTD
    # 21. PI : Publisher City = OXFORD
    # 22. PA : Publisher Address = THE BOULEVARD, LANGFORD LANE, KIDLINGTON, OXFORD OX5 1GB, OXON, ENGLAND
    # 23. SN : International Standard Serial Number (ISSN) = 0959-6526
    sn_ = "None" if ab.issn == None else ab.issn

    # 24. EI : Electronic International Standard Serial Number (eISSN) = 1879-1786
    # 25. J9 : 29-Character Source Abbreviation = J CLEAN PROD
    j9_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"
    j9_ = j9_.upper()

    # 26. JI : ISO Source Abbreviation = J. Clean Prod.
    ji_ = ab.sourcetitle_abbreviation if ab.sourcetitle_abbreviation != None else "None"

    # 27. PD : Publication Date = JUL 1
    month = ab.coverDate.split('-')[1]
    date = ab.coverDate.split('-')[2]
    pd_ = f"{calendar.month_name[int(month)][:3].upper()} {int(date)}"

    # 28. PY : Publication Year = 2020
    py_ = ab.coverDate.split('-')[0]

    # 29. VL : Volumn = 260
    vl_ = ab.volume

    # 30. AR : Article Number = 121059
    ar_ = df_bipv.loc[art, "article_number"] if df_bipv.loc[art, "article_number"] != None else "None"

    # 31. DI : Digital Object Identifier = 10.1016/j.jclepro.2020.121059
    doi_ = ab.doi

    # 32. PG : Page Count = 14
    # 33. WC : Web of Science Categories = Green & Sustainable Science & Technology; Engineering, Environmental; Environmental Sciences
    # 34. SC : Research Areas = Science & Technology - Other Topics; Engineering; Environmental Sciences & Ecology
    if ab.subject_areas:        
        tmp = pd.DataFrame(ab.subject_areas)
        tmp_ = tmp["area"].tolist()
        sc_ = "; ".join(tmp_)
    else: 
        sc_ = [None]

    # 35. GA : Document Delivery Number = LL4XH
    # 36. UT : Accession Number = WOS:000531559900003
    # 37. DA : Date this report was generated. = 2020-06-14
    
    # summation
    data=[eid, docu_type_, index_name_, author_name_, docu_title_, 
                                src_title_, src_abb_, language_, docu_type_, auth_kw_, 
                                kw_plus_, abstract_, addresss_, rep_addr_, em_addr_, 
                                refs_, nr_, tc_, cc_, sn_, 
                                j9_, ji_, pd_, py_, vl_, 
                                ar_, doi_, sc_]
    
    df_ab = df_ab.append(dict(zip(columns, data)), ignore_index=True)

looping over df_bipv: 100%|██████████| 14385/14385 [07:05<00:00, 33.82it/s]


In [7]:
list_rooftop_eid = df_rooftop["eid"].tolist()
df_ab["is_rooftop"] = df_ab["eid"].apply(lambda x: True if x in list_rooftop_eid else False)

In [8]:
df_ab.to_pickle("./210803/df_ab.pkl")
df_ab.shape

(14385, 29)